In [18]:
import requests
import gzip
import json

# URL del archivo JSON comprimido
url = 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/goodreads_reviews_dedup.json.gz'

# Realizar la solicitud GET al servidor
response = requests.get(url, stream=True)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Descomprimir el contenido del archivo
    with gzip.GzipFile(fileobj=response.raw) as f:
        # Leer las primeras 500 filas del JSON
        primeras_500_filas = [json.loads(next(f)[:-1].decode('utf-8')) for _ in range(50000)]

    # Guardar las primeras 500 filas en un nuevo archivo JSON
    with open('dd.json', 'w', encoding='utf-8') as file:
        json.dump(primeras_500_filas, file, indent=2)

    print("Las primeras 500 filas del archivo JSON fueron descargadas exitosamente.")
else:
    print(f"Error al descargar el archivo. Código de estado: {response.status_code}")

Las primeras 500 filas del archivo JSON fueron descargadas exitosamente.


In [19]:
import pandas as pd
# Read eoo.json only user_id, book_id, rating
df = pd.read_json('dd.json')
df = df[['user_id', 'book_id', 'rating']]

In [20]:
print(df.head())

                            user_id   book_id  rating
0  8842281e1d1347389f2ab93d60773d4d  24375664       5
1  8842281e1d1347389f2ab93d60773d4d  18245960       5
2  8842281e1d1347389f2ab93d60773d4d   6392944       3
3  8842281e1d1347389f2ab93d60773d4d  22078596       4
4  8842281e1d1347389f2ab93d60773d4d   6644782       4


In [30]:
# fes un merge entre el dataframe anterior i el 500_llibres.json segons el book_id i guarda el resultat en un nou dataframe
df2 = pd.read_json('500_llibres.json')

# Selecciona els atributs desitjats del dataframe 500_llibres.json

selected_columns = ['user_id', 'book_id', 'rating', 'isbn', 'text_reviews_count', 'series', 'country_code', 'language_code', 'is_ebook', 'average_rating', 'similar_books', 'author_id', 'publisher','num_pages']  # Afegeix els altres atributs desitjats

# Realitza el merge
df_merged = pd.merge(df, df2, on='book_id')

# Selecciona només les columnes desitjades
df_final = df_merged[selected_columns]

# Guarda el resultat en un nou fitxer CSV
df_final.to_csv('resultat_final.csv', index=False)


MergeError: Passing 'suffixes' which cause duplicate columns {'language_code_x', 'authors_x', 'average_rating_x', 'title_x'} is not allowed.

In [26]:
if 'popular_shelves' in df.columns:
    df['popular_shelves'] = df['popular_shelves'].apply(lambda x: [{'count': item['count'], 'name': item['name']} for item in x])
    df['popular_shelves'] = df['popular_shelves'].apply(lambda x: x[1] if len(x) > 1 else None)
else:
    print("The 'popular_shelves' column does not exist in the dataframe.")


The 'popular_shelves' column does not exist in the dataframe.
